<a href="https://colab.research.google.com/github/Eason0227/2023-IMBD-final-round/blob/main/Production%20Line%201%20prediciton%20v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
# from google.colab import drive
# drive.mount('/content/drive/')
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import ExponentialSmoothing
# from statsmodels.tsa.holtwinters import E4xponentialSmoothing
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.impute import KNNImputer
import optuna
from statsmodels.tsa.holtwinters import Holt

from sklearn.svm import SVR

In [ ]:
anormaly_train1 = pd.read_csv('anomaly_train.csv')

for i in range(len(anormaly_train1)):
    if anormaly_train1['oven_id'][i] == '1.00E+00':
        anormaly_train1['oven_id'][i] = '1E0'
    elif anormaly_train1['oven_id'][i] == '2.00E+00':
        anormaly_train1['oven_id'][i] = '2E0'

In [ ]:
# 產線二資料
oven,layer,date,anormaly_total_number = [],[],[],[]
for d in pd.date_range('2021/12/27','2023/02/06'):
  for i in ['2B0','2C0','2D0','2E0','2G0']:
    for j in range(1,20):
      oven.append(i)
      layer.append(j)
      date.append(d)
      anormaly_total_number.append(0)

accumulation = np.zeros(407*5*19)
lamp = np.zeros(407*5*19)

full_data = {"oven_id": oven ,'layer_id':layer,'date':date,"lamp_id":lamp,"anomaly_accumulation_hour":accumulation,"anomaly_total_number":anormaly_total_number }
full_data_1 = pd.DataFrame(full_data)

In [ ]:
# 產線一資料
oven,layer,date,anormaly_total_number = [],[],[],[]
for d in pd.date_range('2021/12/27','2022/09/01'):
  for i in ['1B0','1C0','1D0','1E0','1G0']:
    for j in range(1,20):
      oven.append(i)
      layer.append(j)
      date.append(d)
      anormaly_total_number.append(0)

accumulation = np.zeros(249*5*19)
lamp = np.zeros(249*5*19)

full_data = {"oven_id": oven ,'layer_id':layer,'date':date,"lamp_id":lamp,"anomaly_accumulation_hour":accumulation,"anomaly_total_number":anormaly_total_number }
full_data_2 = pd.DataFrame(full_data)

In [ ]:
all_full_data = pd.concat([full_data_1,full_data_2])

In [ ]:
#建立2021/12/27-2023/02/06有異常的爐和層的資料
# anormaly_train_1 = anormaly_train1[['date','oven_id','layer_id','anomaly_total_number']]
anormaly_train1['date'] = pd.to_datetime(anormaly_train1['date'],format='%Y-%m-%d')

#合併2021/12/27-2023/02/06所有爐和所有層的資料
df = pd.concat([anormaly_train1,all_full_data])
df.drop_duplicates(subset=['date','oven_id','layer_id'], keep='first', inplace=True)
df = df.reset_index().drop(['index'],axis=1)

## 加入新特徵

In [ ]:
accumulation_df = pd.read_csv('accumlation_hour.csv')
accumulation_mean = accumulation_df.groupby(['oven_id'])['accumulation_hour'].mean()
accumulation_max =  accumulation_df.groupby(['oven_id'])['accumulation_hour'].max()
accumulation_min = accumulation_df.groupby(['oven_id'])['accumulation_hour'].min()
accumulation_mean

oven_id
1B0    8160.736842
1C0    7049.114035
1D0    7381.078947
1E0    6724.149123
1G0    2553.850877
2B0    6651.411483
2C0    7146.488038
2D0    7182.669856
2E0    5346.928230
2G0    1195.425837
Name: accumulation_hour, dtype: float64

In [ ]:
power_df = pd.read_csv('power_sep.csv')
def power_setup(accumulation_hour) :
    for i in range(len(power_df)):
        if accumulation_hour <= power_df['accumulation_hour1'][i] :
            break
    return power_df['power_setup(other_lamp)'][i]

## 逐步加入預測值 - 最終測試預測結果

In [ ]:
def creat_feature(oven,days,new_prediction):
    oven_df = df[df['oven_id']== oven ].sort_values(by='date',ascending=True)
    oven2_total_anormal = oven_df.groupby(['date'])['anomaly_total_number'].sum()
    oven_total_anormal = pd.DataFrame(oven2_total_anormal)

    for i in range(27): # 產線一預測後27天，加入後27天的row
        oven_total_anormal.loc[249+i] = np.nan

    if len(new_prediction) != 0:
        for i in range(len(new_prediction)):
            oven_total_anormal.loc[249+i] = new_prediction[i]

    oven_total_anormal['oven_id'] = oven

    # 過去27天的資料
    oven_total_anormal['number_sum'] = np.nan
    oven_total_anormal['number_max'] = np.nan
    oven_total_anormal['number_min'] =  np.nan
    oven_total_anormal['number_mode'] = np.nan
    oven_total_anormal['days'] = np.nan

    # 每個爐的平均損壞數量
    oven_total_anormal['oven_encoder'] = oven_total_anormal['anomaly_total_number'].sum() / np.sum(oven_total_anormal['anomaly_total_number'] >0 )

    # 個別爐的水冷板溫度、累積時數、Power setup
    if oven == '1B0':
        oven_total_anormal['cooler_max'] = 30.9
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[0] ,accumulation_max[0],accumulation_min[0]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[0])

    elif oven == '1C0':
        oven_total_anormal['cooler_max'] = 30.5
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[1],accumulation_max[1],accumulation_min[1]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[1])

    elif oven == '1D0':
        oven_total_anormal['cooler_max'] = 27.5
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[2],accumulation_max[2],accumulation_min[2]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[2])

    elif oven == '1E0':
        oven_total_anormal['cooler_max'] = 27
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[3],accumulation_max[3],accumulation_min[3]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[3])

    elif oven == '1G0':
        oven_total_anormal['cooler_max'] = 30.1
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[4],accumulation_max[4],accumulation_min[4]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[4])

    elif oven == '2B0':
        oven_total_anormal['cooler_max'] = 25.9
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[5] ,accumulation_max[5],accumulation_min[5]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[5])

    elif oven == '2C0':
        oven_total_anormal['cooler_max'] = 25.3
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[6],accumulation_max[6],accumulation_min[6]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[6])

    elif oven == '2D0':
        oven_total_anormal['cooler_max'] = 26
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[7],accumulation_max[7],accumulation_min[7]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[7])

    elif oven == '2E0':
        oven_total_anormal['cooler_max'] = 25.1
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[8],accumulation_max[8],accumulation_min[8]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[8])

    elif oven == '2G0':
        oven_total_anormal['cooler_max'] = 26.3
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[9],accumulation_max[9],accumulation_min[9]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[9])

    # 過去22天的資料
    for i in range(27,len(oven_total_anormal)):
        oven_total_anormal['number_sum'][i] = np.sum(oven_total_anormal['anomaly_total_number'][i-27:i])
        oven_total_anormal['number_max'][i] = np.max(oven_total_anormal['anomaly_total_number'][i-27:i])
        oven_total_anormal['number_min'][i] = np.min(oven_total_anormal['anomaly_total_number'][i-27:i])
        vals,counts = np.unique( oven_total_anormal[i-27:i][ oven_total_anormal['anomaly_total_number'][i-27:i]>0 ]['anomaly_total_number'],return_counts=True)
        try:
            index = np.argmax(counts)
            oven_total_anormal['number_mode'][i] = vals[index]
        except: # 找不到眾數
            oven_total_anormal['number_mode'][i] = 0
        oven_total_anormal['days'][i] = np.sum(oven_total_anormal['anomaly_total_number'][i-27:i] >0 )
    return oven_total_anormal

下面code，有一個小bug，因為最後一天資料跑不出來，所以for迴圈只跑 倒數第27,26,...,3,2共26天的資料，倒數第一天資料是直接倒到數第二天的值。

In [ ]:
lasso_predss = []
for oven_number in['1B0','1C0','1D0','1E0','1G0']:
    print("============================= oven "+oven_number+" =============================")
    lasso_preds = []
    for i in range(27,1,-1):
        prediction_days = i

        train_df = creat_feature(oven='1B0',days = prediction_days, new_prediction = lasso_preds )[:-prediction_days]

        # if prediction_days == 1 :
        #     test_df = creat_feature(oven='1B0',days = prediction_days, new_prediction = lasso_preds)[275:276]
        # else:
        #     test_df = creat_feature(oven='1B0',days = prediction_days, new_prediction = lasso_preds)[-prediction_days: - prediction_days+1 ]
        test_df = creat_feature(oven='1B0',days = prediction_days, new_prediction = lasso_preds)[-prediction_days: - prediction_days+1 ]

        for j in ['1C0','1D0','1E0','1G0','2B0','2C0','2D0','2E0','2G0']:
            training_df = creat_feature(oven = j ,days=prediction_days, new_prediction = lasso_preds )[:-prediction_days]
            # if prediction_days == 1 :
            #     test_df = creat_feature(oven = j ,days = prediction_days, new_prediction = lasso_preds)[275:276]
            # else:
            #     testing_df = creat_feature(oven = j ,days=prediction_days, new_prediction = lasso_preds )[-prediction_days: - prediction_days+1]
            testing_df = creat_feature(oven = j ,days=prediction_days, new_prediction = lasso_preds )[-prediction_days: - prediction_days+1]
            train_df = pd.concat( [train_df, training_df ])
            test_df = pd.concat( [test_df, testing_df ])

        train_x = train_df.drop(['anomaly_total_number','oven_id'],axis=1)
        train_y = train_df['anomaly_total_number']

        test_1 = test_df[test_df['oven_id'] == oven_number ]
        test_1_x = test_1.drop(['anomaly_total_number','oven_id'],axis=1)

        imputed_data =  KNNImputer(n_neighbors=3).fit_transform(train_x)
        lasso_pred = Lasso().fit(imputed_data, train_y).predict(test_1_x)
        lasso_preds.append(lasso_pred)
        # print(i)

    lasso_preds.append(lasso_preds[-1])
    lasso_predss.append(np.round(np.sum(lasso_preds)))
    print('lasso :',lasso_predss[-1])

## 驗證集預測結果

In [ ]:
anormaly_train1 = pd.read_csv('anomaly_train.csv')

for i in range(len(anormaly_train1)):
    if anormaly_train1['oven_id'][i] == '1.00E+00':
        anormaly_train1['oven_id'][i] = '1E0'
    elif anormaly_train1['oven_id'][i] == '2.00E+00':
        anormaly_train1['oven_id'][i] = '2E0'

anormaly_train1['date'] = pd.to_datetime(anormaly_train1['date'],format='%Y-%m-%d')


In [ ]:
import datetime
index = []
for i in range(len(anormaly_train1)):
    if (anormaly_train1['oven_id'][i][0] == '1') and (anormaly_train1['date'][i]  > datetime.date(2022,8,5)) :
            index.append(i)

anormaly_train1 = anormaly_train1.drop( index = index ).reset_index().drop(['index'], axis=1)

In [ ]:
# 產線一資料
oven,layer,date,anormaly_total_number = [],[],[],[]
for d in pd.date_range('2021/12/27','2022/08/05'):
  for i in ['1B0','1C0','1D0','1E0','1G0']:
    for j in range(1,20):
      oven.append(i)
      layer.append(j)
      date.append(d)
      anormaly_total_number.append(0)

accumulation = np.zeros(222*5*19)
lamp = np.zeros(222*5*19)

full_data = {"oven_id": oven ,'layer_id':layer,'date':date,"lamp_id":lamp,"anomaly_accumulation_hour":accumulation,"anomaly_total_number":anormaly_total_number }
full_data_2 = pd.DataFrame(full_data)

all_full_data = pd.concat([full_data_1,full_data_2])

#建立2021/12/27-2023/02/06有異常的爐和層的資料
# anormaly_train_1 = anormaly_train1[['date','oven_id','layer_id','anomaly_total_number']]

#合併2021/12/27-2023/02/06所有爐和所有層的資料
df = pd.concat([anormaly_train1,all_full_data])
df.drop_duplicates(subset=['date','oven_id','layer_id'], keep='first', inplace=True)
df = df.reset_index().drop(['index'],axis=1)

## 逐步預測法

In [ ]:
def creat_feature(oven,days,new_prediction):
    oven_df = df[df['oven_id']== oven ].sort_values(by='date',ascending=True)
    oven2_total_anormal = oven_df.groupby(['date'])['anomaly_total_number'].sum()
    oven_total_anormal = pd.DataFrame(oven2_total_anormal)

    for i in range(27): # 產線一預測後27天，加入後27天的row，並設為nan
        oven_total_anormal.loc[222+i] = np.nan

    if len(new_prediction) != 0: # 將每次的預測值逐步加入資料
        for i in range(len(new_prediction)):
            oven_total_anormal.loc[222+i] = new_prediction[i]

    oven_total_anormal['oven_id'] = oven

    # 過去27天的資料
    oven_total_anormal['number_sum'] = np.nan
    oven_total_anormal['number_max'] = np.nan
    oven_total_anormal['number_min'] =  np.nan
    oven_total_anormal['number_mode'] = np.nan
    oven_total_anormal['days'] = np.nan

    # 每個爐的平均損壞數量
    oven_total_anormal['oven_encoder'] = oven_total_anormal['anomaly_total_number'].sum() / np.sum(oven_total_anormal['anomaly_total_number'] >0 )

    # 個別爐的水冷板溫度、累積時數、Power setup
    if oven == '1B0':
        oven_total_anormal['cooler_max'] = 30.9
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[0] ,accumulation_max[0],accumulation_min[0]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[0])

    elif oven == '1C0':
        oven_total_anormal['cooler_max'] = 30.5
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[1],accumulation_max[1],accumulation_min[1]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[1])

    elif oven == '1D0':
        oven_total_anormal['cooler_max'] = 27.5
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[2],accumulation_max[2],accumulation_min[2]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[2])

    elif oven == '1E0':
        oven_total_anormal['cooler_max'] = 27
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[3],accumulation_max[3],accumulation_min[3]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[3])

    elif oven == '1G0':
        oven_total_anormal['cooler_max'] = 30.1
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[4],accumulation_max[4],accumulation_min[4]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[4])

    elif oven == '2B0':
        oven_total_anormal['cooler_max'] = 25.9
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[5] ,accumulation_max[5],accumulation_min[5]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[5])

    elif oven == '2C0':
        oven_total_anormal['cooler_max'] = 25.3
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[6],accumulation_max[6],accumulation_min[6]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[6])

    elif oven == '2D0':
        oven_total_anormal['cooler_max'] = 26
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[7],accumulation_max[7],accumulation_min[7]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[7])

    elif oven == '2E0':
        oven_total_anormal['cooler_max'] = 25.1
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[8],accumulation_max[8],accumulation_min[8]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[8])

    elif oven == '2G0':
        oven_total_anormal['cooler_max'] = 26.3
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[9],accumulation_max[9],accumulation_min[9]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[9])

    # 過去22天的資料
    for i in range(27,len(oven_total_anormal)):
        oven_total_anormal['number_sum'][i] = np.sum(oven_total_anormal['anomaly_total_number'][i-27:i])
        oven_total_anormal['number_max'][i] = np.max(oven_total_anormal['anomaly_total_number'][i-27:i])
        oven_total_anormal['number_min'][i] = np.min(oven_total_anormal['anomaly_total_number'][i-27:i])
        vals,counts = np.unique( oven_total_anormal[i-27:i][ oven_total_anormal['anomaly_total_number'][i-27:i]>0 ]['anomaly_total_number'],return_counts=True)
        try:
            index = np.argmax(counts)
            oven_total_anormal['number_mode'][i] = vals[index]
        except: # 找不到眾數
            oven_total_anormal['number_mode'][i] = 0
        oven_total_anormal['days'][i] = np.sum(oven_total_anormal['anomaly_total_number'][i-27:i] >0 )
    return oven_total_anormal

下面code，有一個小bug，因為最後一天資料跑不出來，所以for迴圈只跑 倒數第27,26,...,3,2共26天的資料，倒數第一天資料是直接倒到數第二天的值。

In [ ]:
lasso_predss = []
for oven_number in['1B0','1C0','1D0','1E0','1G0']:
    print("============================= oven "+oven_number+" =============================")
    lasso_preds = []
    for i in range(27,1,-1):
        prediction_days = i

        train_df = creat_feature(oven='1B0',days = prediction_days, new_prediction = lasso_preds )[:-prediction_days]

        # if prediction_days == 1 :
        #     test_df = creat_feature(oven='1B0',days = prediction_days, new_prediction = lasso_preds)[248:249]
        # else:
        #     test_df = creat_feature(oven='1B0',days = prediction_days, new_prediction = lasso_preds)[-prediction_days: - prediction_days+1 ]
        test_df = creat_feature(oven='1B0',days = prediction_days, new_prediction = lasso_preds)[-prediction_days: - prediction_days+1 ]

        for j in ['1C0','1D0','1E0','1G0','2B0','2C0','2D0','2E0','2G0']:
            training_df = creat_feature(oven = j ,days=prediction_days, new_prediction = lasso_preds )[:-prediction_days]
            # if prediction_days == 1 :
            #     test_df = creat_feature(oven = j ,days = prediction_days, new_prediction = lasso_preds)[248:249]
            # else:
            #     testing_df = creat_feature(oven = j ,days=prediction_days, new_prediction = lasso_preds )[-prediction_days: - prediction_days+1]
            testing_df = creat_feature(oven = j ,days=prediction_days, new_prediction = lasso_preds )[-prediction_days: - prediction_days+1]
            train_df = pd.concat( [train_df, training_df ])
            test_df = pd.concat( [test_df, testing_df ])

        train_x = train_df.drop(['anomaly_total_number','oven_id'],axis=1)
        train_y = train_df['anomaly_total_number']

        test_1 = test_df[test_df['oven_id'] == oven_number ]
        test_1_x = test_1.drop(['anomaly_total_number','oven_id'],axis=1)

        imputed_data =  KNNImputer(n_neighbors=3).fit_transform(train_x)
        lasso_pred = Lasso().fit(imputed_data, train_y).predict(test_1_x)
        lasso_preds.append(lasso_pred)
        # print(i)

    lasso_preds.append(lasso_preds[-1])
    lasso_predss.append(np.round(np.sum(lasso_preds)))
    print('lasso :',lasso_predss[-1])

============================= oven 1B0 =============================
lasso : 60.0
============================= oven 1C0 =============================
lasso : 43.0
============================= oven 1D0 =============================
lasso : 39.0
============================= oven 1E0 =============================
lasso : 35.0
============================= oven 1G0 =============================
lasso : 3.0


In [ ]:
## 使用catboost
lasso_predss = []
for oven_number in['1B0','1C0','1D0','1E0','1G0']:
    print("============================= oven "+oven_number+" =============================")
    lasso_preds = []
    for i in range(27,1,-1):
        prediction_days = i

        train_df = creat_feature(oven='1B0',days = prediction_days, new_prediction = lasso_preds )[:-prediction_days]

        if prediction_days == 1 :
            test_df = creat_feature(oven='1B0',days = prediction_days, new_prediction= lasso_preds)[248:249]
        else:
            test_df = creat_feature(oven='1B0',days = prediction_days, new_prediction= lasso_preds)[-prediction_days: - prediction_days+1 ]

        for j in ['1C0','1D0','1E0','1G0','2B0','2C0','2D0','2E0','2G0']:
            training_df = creat_feature(oven = j ,days=prediction_days, new_prediction = lasso_preds )[:-prediction_days]
            if prediction_days == 1 :
                test_df = creat_feature(oven = j ,days = prediction_days, new_prediction= lasso_preds)[248:249]
            else:
                testing_df = creat_feature(oven = j ,days=prediction_days, new_prediction = lasso_preds )[-prediction_days: - prediction_days+1]
            train_df = pd.concat( [train_df, training_df ])
            test_df = pd.concat( [test_df, testing_df ])

        train_x = train_df.drop(['anomaly_total_number','oven_id'],axis=1)
        train_y = train_df['anomaly_total_number']

        test_1 = test_df[test_df['oven_id'] == oven_number ]
        test_1_x = test_1.drop(['anomaly_total_number','oven_id'],axis=1)

        imputed_data =  KNNImputer(n_neighbors=3).fit_transform(train_x)
        lasso_pred = CatBoostRegressor(silent=True,random_state=42).fit(imputed_data, train_y).predict(test_1_x)
        lasso_preds.append(lasso_pred)
        # print(i)

    lasso_preds.append(lasso_preds[-1])
    lasso_predss.append(np.round(np.sum(lasso_preds)))
    print('CatBoost :',lasso_predss[-1])

## 正常預測法

In [ ]:
def creat_feature(oven,days):

    oven_df = df[df['oven_id']== oven ].sort_values(by='date',ascending=True)
    oven2_total_anormal = oven_df.groupby(['date'])['anomaly_total_number'].sum()
    oven_total_anormal = pd.DataFrame(oven2_total_anormal)

    for i in range(27): # 產線一預測後27天，加入後27天的row
        oven_total_anormal.loc[222+i] = np.nan

    oven_total_anormal['oven_id'] = oven

    # 過去27天的資料
    oven_total_anormal['number_sum'] = np.nan
    oven_total_anormal['number_max'] = np.nan
    oven_total_anormal['number_min'] =  np.nan
    oven_total_anormal['number_mode'] = np.nan
    oven_total_anormal['days'] = np.nan

    # 每個爐的平均損壞數量
    oven_total_anormal['oven_encoder'] = oven_total_anormal['anomaly_total_number'].sum() / np.sum(oven_total_anormal['anomaly_total_number'] >0 )

    # 個別爐的水冷板溫度、累積時數、Power setup
    if oven == '1B0':
        oven_total_anormal['cooler_max'] = 30.9
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[0] ,accumulation_max[0],accumulation_min[0]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[0])

    elif oven == '1C0':
        oven_total_anormal['cooler_max'] = 30.5
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[1],accumulation_max[1],accumulation_min[1]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[1])

    elif oven == '1D0':
        oven_total_anormal['cooler_max'] = 27.5
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[2],accumulation_max[2],accumulation_min[2]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[2])

    elif oven == '1E0':
        oven_total_anormal['cooler_max'] = 27
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[3],accumulation_max[3],accumulation_min[3]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[3])

    elif oven == '1G0':
        oven_total_anormal['cooler_max'] = 30.1
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[4],accumulation_max[4],accumulation_min[4]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[4])

    elif oven == '2B0':
        oven_total_anormal['cooler_max'] = 25.9
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[5] ,accumulation_max[5],accumulation_min[5]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[5])

    elif oven == '2C0':
        oven_total_anormal['cooler_max'] = 25.3
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[6],accumulation_max[6],accumulation_min[6]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[6])

    elif oven == '2D0':
        oven_total_anormal['cooler_max'] = 26
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[7],accumulation_max[7],accumulation_min[7]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[7])

    elif oven == '2E0':
        oven_total_anormal['cooler_max'] = 25.1
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[8],accumulation_max[8],accumulation_min[8]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[8])

    elif oven == '2G0':
        oven_total_anormal['cooler_max'] = 26.3
        oven_total_anormal['avg_accumulation'] ,oven_total_anormal['max_accumulation'] ,oven_total_anormal['min_accumulation'] = accumulation_mean[9],accumulation_max[9],accumulation_min[9]
        oven_total_anormal['avg_power_setup'] = power_setup(accumulation_mean[9])

    # 過去22天的資料
    for i in range(days,len(oven_total_anormal)):
        oven_total_anormal['number_sum'][i] = np.sum(oven_total_anormal['anomaly_total_number'][i-days:i])
        oven_total_anormal['number_max'][i] = np.max(oven_total_anormal['anomaly_total_number'][i-days:i])
        oven_total_anormal['number_min'][i] = np.min(oven_total_anormal['anomaly_total_number'][i-days:i])
        vals,counts = np.unique( oven_total_anormal[i-days:i][ oven_total_anormal['anomaly_total_number'][i-days:i]>0 ]['anomaly_total_number'],return_counts=True)
        try:
            index = np.argmax(counts)
            oven_total_anormal['number_mode'][i] = vals[index]
        except: # 找不到眾數
            oven_total_anormal['number_mode'][i] = 0
        oven_total_anormal['days'][i] = np.sum(oven_total_anormal['anomaly_total_number'][i-days:i] >0 )
    return oven_total_anormal

def rmse(y,yhat):
  return np.sqrt(mean_squared_error(y,yhat))

In [ ]:
prediction_days = 27

train_df = creat_feature(oven='1B0',days=prediction_days)[:-prediction_days]
test_df = creat_feature(oven='1B0',days=prediction_days)[-prediction_days:]

for i in ['1C0','1D0','1E0','1G0','2B0','2C0','2D0','2E0','2G0']:
    training_df = creat_feature(oven=i,days=prediction_days)[:-prediction_days]
    testing_df = creat_feature(oven=i,days=prediction_days)[-prediction_days:]
    train_df = pd.concat( [train_df, training_df ])
    test_df = pd.concat( [test_df, testing_df ])

train_x = train_df.drop(['anomaly_total_number','oven_id'],axis=1)
train_y = train_df['anomaly_total_number']

In [ ]:
lasso_preds,en_preds,lgb_preds,xgb_preds,cat_preds,actual = [],[],[],[],[],[]
lgb_params = {'n_estimators': 900, 'learning_rate': 0.08108549380709631, 'reg_lambda': 7.400420560246068, 'subsample': 0.6262267316079506, 'min_child_samples': 7, 'max_depth': 9}
xgb_params = {'n_estimators': 1300, 'learning_rate': 0.024658403720628174, 'booster': 'gbtree', 'lambda': 8.65039444200474, 'alpha': 7.841025757534408, 'subsample': 0.9651727299534355, 'colsample_bytree': 0.8353388492801248, 'max_depth': 8, 'min_child_weight': 3, 'eta': 0.8339832871331261, 'gamma': 0.6596571553899819, 'grow_policy': 'lossguide'}
cat_params = {'iterations': 600, 'learning_rate': 0.08077046387065928, 'depth': 7, 'l2_leaf_reg': 9.566779128474014, 'bagging_temperature': 3.857521071572685, 'random_strength': 4.330651960392663, 'border_count': 47}

for i in ['1B0','1C0','1D0','1E0','1G0']:
# for i in ['2B0','2C0','2D0','2E0','2G0']:
    print("============================= Test prediction on oven "+i+" =============================")
    test_1 = test_df[test_df['oven_id']==i]
    test_1_x = test_1.drop(['anomaly_total_number','oven_id'],axis=1)

    # 执行KNN插补
    imputed_data =  KNNImputer(n_neighbors=3).fit_transform(train_x)
    # imputed_data = train_x.fillna(0)

    lasso_pred = Lasso().fit(imputed_data, train_y).predict(test_1_x)
    en_pred = ElasticNet().fit(imputed_data, train_y).predict(test_1_x)
    xgb_pred = XGBRegressor(random_state= 42,**xgb_params).fit(train_x, train_y).predict(test_1_x)
    lgb_pred = LGBMRegressor(random_state= 42,**lgb_params).fit(train_x, train_y).predict(test_1_x)
    cat_pred = CatBoostRegressor(random_state= 42,silent=True).fit(train_x, train_y).predict(test_1_x)

    lasso_preds.append(np.round(np.sum(lasso_pred)))
    en_preds.append(np.round(np.sum(en_pred)))
    lgb_preds.append(np.round(np.sum(lgb_pred)))
    xgb_preds.append(np.round(np.sum(xgb_pred)))
    cat_preds.append(np.round(np.sum(cat_pred)))
    print('lasso :',lasso_preds[-1],'| ElasticNet :', en_preds[-1],'| LGBM :',lgb_preds[-1],'| XGB :',xgb_preds[-1],'| Catboost : ',cat_preds[-1])

============================= Test prediction on oven 1B0 =============================
lasso : 32.0 | ElasticNet : 31.0 | LGBM : 60.0 | XGB : 46.0 | Catboost :  57.0
============================= Test prediction on oven 1C0 =============================
lasso : 30.0 | ElasticNet : 29.0 | LGBM : 16.0 | XGB : 53.0 | Catboost :  73.0
============================= Test prediction on oven 1D0 =============================
lasso : 31.0 | ElasticNet : 31.0 | LGBM : 68.0 | XGB : 55.0 | Catboost :  49.0
============================= Test prediction on oven 1E0 =============================
lasso : 30.0 | ElasticNet : 30.0 | LGBM : 93.0 | XGB : 47.0 | Catboost :  113.0
============================= Test prediction on oven 1G0 =============================
lasso : 3.0 | ElasticNet : 2.0 | LGBM : 1.0 | XGB : 1.0 | Catboost :  1.0
